In [24]:
# Load all image paths
import pandas as pd
dataset_path = '../Datasets/CIFAR10'
data_df = pd.read_csv(dataset_path + '/data.csv')
SEED = 20

In [25]:
import random
random.seed(SEED)

In [26]:
# Split test dataset and private (data owner) dataset
test_dataset = data_df.groupby('label').sample(frac=.2) 
private_dataset = data_df.iloc[[i for i in data_df.index if i not in test_dataset.index]]

In [27]:
# Data owner A: Plane, Car, Ship, Boat, Truck, Other
round(random.randint(5, 20)) 

9

In [3]:
# 
# get_dataset()

In [ ]:
# test_dataset = 20%
# private_dataset = 80%


In [ ]:
# Dataset Partition and Simulation
# Test Dataset: 20%
# Training baseline model
# Model evaluation
# Private Dataset: 80%
# Number per class: random hundreds between 500~3000
# Data Owners (10)
# A: Plane, Car, Ship, Boat, Truck, Other
# B: Truck, Car, Other
# C: Plane, Truck, Ship, Other
# D: Ship, Plane
# E: Plane, Car, Ship, Other (overlapping dataset)
# F: Cat, Dog, Other (non existing labels)
# G: Truck, Other (drop below baseline models)
# T1: Truck, Car (all labels below baseline)
# T2: Plane, Boat (one label below baseline)
# T3: Cat, Dog (large model)
# Clients (4)
# X: Truck
# Y: Truck (select models above baseline)
# Z: Plane, Ship (multiple labels)
# T4: Horse (non existing)
